In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse
from time import sleep, time
from random import randint
import re
from IPython.core.display import clear_output
from warnings import warn
import numpy as np
from selenium import webdriver 
import cv2 
import pytesseract
from PIL import Image

In [2]:
def is_captcha_page(soup):
    captcha_text="Please verify you are human by typing the text below and clicking submit."
    page_info=soup.find('font',attrs={'class':"pagebodybold"})
    if page_info!=None and captcha_text in page_info.text:
        return True
    return False

In [3]:
def solve_captcha(company_link):
    driver = webdriver.Firefox('/home/udit/projects/chromiumm')
    driver.get(company_link)
    element = driver.find_element_by_id("imgCaptcha")
    element.screenshot('final.png')
    img = cv2.imread('final.png')
    captcha=pytesseract.image_to_string(img)
    
    captcha=re.sub(r'\W+', '', captcha)
    captcha_input = driver.find_element_by_xpath("//input[@id ='txtCaptcha']") 
    captcha_input.send_keys(captcha) 

    # submit button clicked
    driver.find_element_by_xpath("//input[@id ='btnRegiser']").click()
    html = driver.page_source
    soup = BeautifulSoup(html)
    driver.close()
    return soup

In [35]:
def try_filling(strr,company_info):
    try:
        exec("print(strr)")
        return exec("return strr")
    except IndexError:
        return(np.nan)

In [5]:
URL = "https://ibanknet.com/scripts/callreports/fiList.aspx?type=secbanks"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [6]:
companies=soup.find_all('a', attrs = {'class':'pagebody'}) 

In [7]:
all_companies=[]

for company in companies:
    company_link="https://ibanknet.com/"+company.get('href')
    c = requests.get(company_link)
    soup = BeautifulSoup(c.content, 'html5lib')
    
    while(is_captcha_page(soup)):
        soup=solve_captcha(company_link)
        
    company_info=[]

    for info in soup.find_all('td', attrs = {'class':'instinfo'}):
        company_info.append(info.text)


    company_inf={}
    company_inf["company_name"]=soup.find('font',attrs={'class':"blockuhilite"}).text.strip()
    company_inf["RSSD_ID"]=company_info[0].split()[company_info[0].split().index("RSSD")+2]
    company_inf['RefinedAddress']=company_info[1]
    company_inf['City']=company_info[4].split(",")[0].strip()
    company_inf['State']=company_info[4].split(",")[1].strip()
    company_inf['Pin']=try_filling('company_info[4].split(",")[2].strip()',company_info)
    company_inf['webAddress']=company_info[7].strip() if ('.com' in company_info[7].strip()) else np.nan
    company_inf['EntityType']=company_info[company_info.index('Entity Type : ')+1]
    company_inf['Incorporated']=company_info[company_info.index('Incorporated : ')+1]
    company_inf['SIC_Code']=company_info[company_info.index('SIC Code : ')+1]
    #company_inf['FederalRegulator']=company_info[company_info.index('Federal Regulator :')+1]
    company_inf['FiscalYearEnd']=company_info[company_info.index('Fiscal Year End : ')+1]
    company_inf['Regulator']=company_info[company_info.index('Regulator : ')+1]
    company_inf['FilerStatus']=company_info[company_info.index('Filer Status : ')+1]
    
    all_companies.append(company_inf)

NoSuchWindowException: Message: Browsing context has been discarded


In [8]:
all_compaies=pd.DataFrame(all_companies)

In [9]:
all_compaies.head()

,company_name,RSSD_ID,RefinedAddress,City,State,Pin,webAddress,EntityType,Incorporated,SIC_Code,FiscalYearEnd,Regulator,FilerStatus
0,JPMORGAN CHASE & CO,1039502,383 MADISON AVENUE,NEW YORK,NY,None,NaN,NATIONAL COMMERCIAL BANKS,"DE, US",6021,1231,SEC,Large Accelerated
1,BANK OF AMERICA CORP /DE/,1073757,BANK OF AMERICA CORPORATE CENTER,CHARLOTTE,NC,None,NaN,NATIONAL COMMERCIAL BANKS,"DE, US",6021,1231,SEC,Large Accelerated
2,CITIGROUP INC,1951350,388 GREENWICH STREET,NEW YORK,NY,None,NaN,NATIONAL COMMERCIAL BANKS,"DE, US",6021,1231,SEC,Large Accelerated
3,WELLS FARGO & COMPANY/MN,1120754,420 MONTGOMERY STREET,SAN FRANCISCO,CA,None,NaN,NATIONAL COMMERCIAL BANKS,"DE, US",6021,1231,SEC,Large Accelerated
4,US BANCORP DE,1119794,U.S. BANCORP,MINNEAPOLIS,MN,None,NaN,NATIONAL COMMERCIAL BANKS,"DE, US",6021,1231,SEC,Large Accelerated


In [10]:
company_info

['CIK 927628 | RSSD ID 2277860 | EIN 541719854',
 '1680 CAPITAL ONE DRIVE',
 'Period : ',
 '2020-03-312019-12-312019-09-302019-06-302019-03-31',
 'MCLEAN, VA, 22102',
 'Filed : ',
 '2020-05-01',
 'US',
 'Document : ',
 '10-Q',
 'Incorporated : ',
 'DE, US',
 'Filer Status : ',
 'Large Accelerated',
 'SIC Code : ',
 '6021',
 'Regulator : ',
 'SEC',
 'Entity Type : ',
 'NATIONAL COMMERCIAL BANKS',
 'Fiscal Year End : ',
 '1231',
 'Ticker : ',
 'COF',
 '',
 '\n\n\n\nRelated Institution\nFederal Regulator\n\n\n \xa0CAPITAL ONE FINANCIAL CORPORATION\nFED\n\n\n \xa0Capital One, National Association\nOCC\n\n\n \xa0Capital One Bank (USA), National Association\nOCC\n\n\n\n',
 'IS - Income Statement',
 'BS - Balance Sheet',
 'EQ - Equity',
 'CF - Cash Flow',
 'CIK 927628 | RSSD ID 2277860 | EIN 541719854',
 '1680 CAPITAL ONE DRIVE',
 'Period : ',
 '2020-03-312019-12-312019-09-302019-06-302019-03-31',
 'MCLEAN, VA, 22102',
 'Filed : ',
 '2020-05-01',
 'US',
 'Document : ',
 '10-Q',
 'Incorporated

In [36]:
a=try_filling(company_info[4].split(",")[2].strip(),company_info)

22102


In [37]:
a

In [38]:
a